# Time series data set (with PyQrack quantum associative memory)

"PyQrack" is a pure Python language standard wrapper for the (C++11) Qrack quantum computer simulator library. PyQrack exposes a "quantum neuron" called "`QrackNeuron`." (Its API reference is [here](https://pyqrack.readthedocs.io/en/latest/autoapi/pyqrack/qrack_neuron/index.html).) We'd like to model a simple data set to achieve a proof-of-concept of using `QrackNeuron`.

In [1]:
#!pip install pyqrack

First, load the data set into a `pandas` dataframe.

In [2]:
import math
import numpy as np
import pandas as pd

df = pd.read_csv('time-series/time-series.csv')
df['Date'] = pd.to_datetime(df['Date'], format='%d-%m-%Y').apply(lambda x:x.toordinal())
# df = df.dropna()

print(df.head())
print("Number of observations: ", df.shape[0])

   Store    Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  733808    1643690.90             0        42.31       2.572   
1      1  733815    1641957.44             1        38.51       2.548   
2      1  733822    1611968.17             0        39.93       2.514   
3      1  733829    1409727.59             0        46.63       2.561   
4      1  733836    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  
Number of observations:  6435


Separate the dependent and independent variables.

In [3]:
keys = ['Holiday_Flag', 'Store', 'Weekly_Sales', 'Temperature', 'Fuel_Price']
dep_key = 'Date'

X = df[keys]
y = df[dep_key] 

In [4]:
train=df.sample(frac = 1/2)
test=df.drop(train.index)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

X = train[keys]
y = train[dep_key]

X_test = test[keys]
y_test = test[dep_key]

Ideally, we'd like to make an improvement on the goodness-of-fit of multiple linear regression, with PyQrack's `QrackNeuron`. At the very least, to show that `QrackNeuron` can be viable for modeling a data set, we'd like to show somehwat comparable performance to multiple linear regression.

PyQrack's `QrackNeuron` can only work with discrete, binary data. To model this or any data set, we have to reduce it to a simple, discrete, binary form.

We'll try to model the data set via "(quantum) associative memory." There are several statistical considerations, to avoid overfit.

Firstly, each possible discretized independent variable permutation input trains an independent parameter of a `QrackNeuron`. If a `QrackNeuron` has never seen a specific, exact permutation of input bits, it has no information about them at all, so its prediction defaults to "maximal superposition," (i.e. a totally random guess). Therefore, we'd like to keep our number of possible distinct inputs significantly fewer in number than our observation rows, when we discretize our indepedent variables.

Satisfying the first consideration, we secondly discretize our dependent variable to have exactly as many possible discrete values as possible distinct inputs. (We guess that this loses the least information about the dependent variable, while we still have enough observations to fully train our network.)

Thirdly, our learning rate should should just barely "saturate" the learned parameters of our (quantum) associative memory. As a learning volatility parameter ("`eta`") of `1/2` "fully trains" one parameter of a `QrackNeuron` between input qubits and output qubit, on average, this implies that we might set `eta` to `1/2` times `2` to the power of input qubits (summed across all predictors) divided by the number of observations. 

At a baseline, our first choice to model this data set might be multiple linear regression.

In [5]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X, y)
pd.DataFrame(zip(X.columns, regr.coef_))

,0,1
0,Holiday_Flag,60.268998
1,Store,-0.901014
2,Weekly_Sales,-0.000011
3,Temperature,0.565100
4,Fuel_Price,485.606124


In [6]:
y_pred = regr.predict(X_test)
sst = 0
ssr = 0
for i in range(len(y_pred)):
    sst += y_test[i] * y_test[i]
    ssr += (y_test[i] - y_pred[i]) * (y_test[i] - y_pred[i])

print("Multiple linear regression validation R^2: ", 1 - ssr / sst)

Multiple linear regression validation R^2:  0.9999999389516051


To discretize the data, we split it into as many quantiles as `2` to the power of our number of input qubits.

In [7]:
store_qubit_count = 6
time_qubit_count = 2
in_qubit_count = 3

time_bin_count = 1 << time_qubit_count
in_key_count = len(keys)
in_bin_count = 1 << in_qubit_count
in_tot_count = 1 + store_qubit_count + (in_key_count - 2) * in_qubit_count

x_bins = []
for i in range(2, len(keys)):
    key = keys[i]
    x_bins.append(np.percentile(X[key], np.arange(0, 100, 100 / in_bin_count)))
y_bins = np.percentile(y, np.arange(0, 100, 100 / time_bin_count))

In [8]:
print(x_bins)

[array([ 215359.21   ,  419651.82625,  551809.05   ,  752488.02625,
        955465.34   , 1184944.37875, 1412790.78   , 1808226.4225 ]), array([-2.06   , 36.75375, 47.55   , 55.075  , 62.725  , 68.88625,
       74.64   , 82.4475 ]), array([2.514  , 2.753  , 2.938  , 3.16275, 3.46   , 3.604  , 3.743  ,
       3.891  ])]


In [9]:
print(y_bins)

[733808. 734060. 734312. 734557.]


Once we have our quantiles, we bin our indepedent training and validation data.

In [10]:
def discretize_x(X):
    xd = []
    for i in X.index:
        xd_row = []
        xd_row.append(X['Holiday_Flag'][i] == 1)
        for p in range(store_qubit_count):
            xd_row.append(((i >> p) & 1) == 1)
        for ki in range(1, len(keys)):
            key = keys[ki]
            bn = in_bin_count
            offset = 0
            while bn > 1:
                bn =  bn // 2
                b = bn + offset
                if X[key][i] < x_bins[ki - 2][b - 1]:
                    xd_row.append(False)
                else:
                    xd_row.append(True)
                    offset += bn
        xd.append(xd_row)
    return xd

xd = discretize_x(X)
xd_test = discretize_x(X_test)

We do the same for our dependent data.

In [11]:
def discretize_y(y):
    yd = []
    for i in y.index:
        yd_row = []
        bn = time_bin_count
        offset = 0
        while bn > 0:
            bn =  bn // 2
            b = bn + offset
            if y[i] < y_bins[b - 1]:
                yd_row.append(False)
            else:
                yd_row.append(True)
                offset += bn
        yd.append(yd_row)
    return yd

yd = discretize_y(y)
yd_test = discretize_y(y_test)

We're ready to train our associative memory! (Note that it offers us no particular advantage, in this case, that our "neurons" are based on simulated quantum computational gates, though it is possible to predict in "superposition" of many rows at once.)

In [12]:
from IPython.display import clear_output
from pyqrack import QrackSimulator, QrackNeuron

eta = (1 / 2) * (time_bin_count / y.shape[0])
input_indices = list(range(in_tot_count))
qsim = QrackSimulator(in_tot_count + time_qubit_count)

qft_qubits = {}
qft_qubits[keys[0]] = [0]
qft_qubits[keys[1]] = []
for i in range(2, len(keys)):
    qft_qubits[keys[i]] = list(range(i * in_qubit_count, (i + 1) * in_qubit_count))

output_layer = []
for i in range(time_qubit_count):
    output_layer.append(QrackNeuron(qsim, input_indices, in_tot_count + i))

# Train the network to associate powers of 2 with their log2()
print("Learning...")
for i in range(len(xd)):
    if ((i + 1) % 100) == 0:
        if (i + 1) != 100:
            clear_output(wait=True)
        print("Epoch ", (i + 1), " out of ", len(xd))
    
    perm = xd[i]
    res = yd[i]

    for j in range(time_qubit_count):
        qsim.reset_all()
        for k in range(in_tot_count):
            if perm[k]:
                qsim.x(k)
        # Transform time domain to Fourier basis
        # for key in keys:
        #     qsim.iqft(qft_qubits[key])
        output_layer[j].learn_permutation(eta, res[j] == 1)

Epoch  3200  out of  3218


Let's use our neural net, trained on half the data, to try to predict the left-out half of data!

In [13]:
print("Should associate each input with its trained output...")
sum_sqr_tot = 0
sum_sqr_res = 0
for i in range(len(xd_test)):
    if ((i + 1) % 100) == 0:
        if (i + 1) != 100:
            clear_output(wait=True)
        print("Predicting ", (i + 1), " out of ", len(xd_test))
    
    perm = xd_test[i]

    qsim.reset_all()
    for j in range(in_tot_count):
        if perm[j]:
            qsim.x(j)
        # Transform time domain to Fourier basis
        # for key in keys:
        #     qsim.iqft(qft_qubits[key])

    for j in range(time_qubit_count):
        output_layer[j].predict()

    bn = time_bin_count
    offset = 0
    for j in range(time_qubit_count):
        bn = bn // 2
        offset += bn * qsim.prob(in_tot_count + j) 
    pred = y_bins[round(offset)]

    sum_sqr_tot += y_test[i] * y_test[i]
    sum_sqr_res += (y_test[i] - pred) * (y_test[i] - pred)

Predicting  3200  out of  3217


How does this compare to the validation R^2 of multiple linear regression?

In [14]:
print("QrackNeuron training R^2: ", 1 - sum_sqr_res / sum_sqr_tot)
print("Multiple linear regression training R^2: ", 1 - ssr / sst)
if (1 - sum_sqr_res / sum_sqr_tot) > (1 - ssr / sst):
    print("QrackNeuron quantum associative memory outperformed multiple linear regression.")
else:
    print("Multiple linear regression outperformed QrackNeuron quantum associative memory.")

QrackNeuron training R^2:  0.9999998049609718
Multiple linear regression training R^2:  0.9999999389516051
Multiple linear regression outperformed QrackNeuron quantum associative memory.
